In [1]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', None)

In [2]:
daily = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/daily_fitbit_sema_df_unprocessed.csv")
hourly = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/hourly_fitbit_sema_df_unprocessed.csv")
breq = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/breq.csv")
panas = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/panas.csv")
personality = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/personality.csv")
stai = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/stai.csv")
ttm = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/ttm.csv")

/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_28262/255447001.py:2: DtypeWarning: Columns (5,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/hourly_fitbit_sema_df_unprocessed.csv")


In [3]:
# preprocessing the daily data
daily.drop("Unnamed: 0", axis=1)

# all values will be associated with the ID, let's count how many there are for each
num_ids = daily['id'].nunique() # 71

# is there an even distribution of dates for the number of records we have (does each ID have the same set of dates?)
# subset = daily.loc[daily['id'].isin(["621e2e8e67b776a24055b564"])]
subset = daily.groupby("id").nunique()
subset = subset.reset_index()
date_per_id_counts = subset[['id', 'date']]

In [4]:
nan_values = daily.isna()
nan_values = nan_values.reset_index()
true_nan_values = nan_values.sum()
true_nan_values

index                               27450345
Unnamed: 0                                 0
id                                         0
date                                       0
nightly_temperature                     4051
nremhr                                  4935
rmssd                                   4935
spo2                                    6140
full_sleep_breathing_rate               4915
stress_score                            5534
sleep_points_percentage                 5534
exertion_points_percentage              5534
responsiveness_points_percentage        5534
daily_temperature_variation             4106
badgeType                               6895
calories                                 750
filteredDemographicVO2Max               2617
distance                                2633
activityType                            5197
bpm                                     2606
lightly_active_minutes                   327
moderately_active_minutes                327
very_activ

In [5]:
ids = list(hourly['id'].unique())

In [6]:
hourly = hourly.drop('Unnamed: 0', axis=1)

In [7]:
all_cols = ["id", "date", "hour","distance", "steps", "calories", "age", "bpm", "bmi", "gender",
             "mindfulness_session", "SAD", "TIRED", "TENSE/ANXIOUS", "ENTERTAINMENT", "GYM", "HOME",
             "OUTDOORS"]
numeric_coerce = ["distance", "steps", "calories","mindfulness_session", "SAD", "TIRED", "TENSE/ANXIOUS", "ENTERTAINMENT", "GYM", "HOME",
             "OUTDOORS"]
tmp = hourly[all_cols]
tmp[numeric_coerce] = tmp[numeric_coerce].apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.int64)
# first 35 ids are good, fails on 36 and after that for some reason

# test_id_subset = ids[36]
# tmp = tmp.loc[hourly['id'] == test_id_subset]

/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_28262/3750693823.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[numeric_coerce] = tmp[numeric_coerce].apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.int64)


In [8]:
roll_up = tmp.groupby(["id", "date"]).agg({
    "distance": ['sum'],
    "steps": ['sum'],
    "calories": ['sum'],
    "gender": ['first'],
    'age': ['first'],
    # 'bmi': ['max'],
    "mindfulness_session": ['max'],
    "SAD": ['max'],
    "TIRED": ["max"],
    "TENSE/ANXIOUS": ['max'],
    "ENTERTAINMENT": ['max'],
    "GYM": ["max"],
    "HOME": ['max'],
    "OUTDOORS": ['max']
    })
roll_up.reset_index()
roll_up


distance  steps calories  gender   age  \
                                         sum    sum      sum   first first   
id                       date                                                
621e2e8e67b776a24055b564 2021-05-24     6509   8833     2339    MALE   <30   
                         2021-05-25     7170   9727     2317    MALE   <30   
                         2021-05-26     6084   8253     2251    MALE   <30   
                         2021-05-27     6644   9015     2314    MALE   <30   
                         2021-05-28     9550  12949     2575    MALE   <30   
...                                      ...    ...      ...     ...   ...   
621e375b67b776a240290cdc 2021-08-13        0      0     1296  FEMALE   <30   
                         2021-08-14        0      0     1296  FEMALE   <30   
                         2021-08-15        0      0     1296  FEMALE   <30   
                         2021-08-16        0      0     1296  FEMALE   <30   
                         2021-08-17        0      0      635  FEMALE   <30   

                                    mindfulness_session SAD TIRED  \
                                                    max max   max   
id                       date                                       
621e2e8e67b776a24055b564 2021-05-24                   0   0     0   
                         2021-05-25                   0   0     1   
                         2021-05-26                   0   0     0   
                         2021-05-27                   0   0     0   
                         2021-05-28                   0   0     0   
...                                                 ...  ..   ...   
621e375b67b776a240290cdc 2021-08-13                   0   0     0   
                         2021-08-14                   0   0     0   
                         2021-08-15                   0   0     0   
                         2021-08-16                   0   0     0   
                         2021-08-17                   0   0     0   

                                    TENSE/ANXIOUS ENTERTAINMENT GYM HOME  \
                                              max           max max  max   
id                       date                                              
621e2e8e67b776a24055b564 2021-05-24             0             0   0    0   
                         2021-05-25             0             0   0    0   
                         2021-05-26             0             0   0    0   
                         2021-05-27             0             0   0    1   
                         2021-05-28             0             0   0    0   
...                                           ...           ...  ..  ...   
621e375b67b776a240290cdc 2021-08-13             0             0   0    0   
                         2021-08-14             0             0   0    0   
                         2021-08-15             0             0   0    0   
                         2021-08-16             0             0   0    0   
                         2021-08-17             0             0   0    0   

                                    OUTDOORS  
                                         max  
id                       date                 
621e2e8e67b776a24055b564 2021-05-24        0  
                         2021-05-25        1  
                         2021-05-26        1  
                         2021-05-27        1  
                         2021-05-28        1  
...                                      ...  
621e375b67b776a240290cdc 2021-08-13        0  
                         2021-08-14        0  
                         2021-08-15        0  
                         2021-08-16        0  
                         2021-08-17        0  

[7005 rows x 13 columns]

In [9]:
tmp['age'].unique()

array(['<30', '>=30', nan], dtype=object)

In [10]:
daily = daily.drop('Unnamed: 0', axis=1)
daily

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,step_goal,min_goal,max_goal,step_goal_label,ALERT,HAPPY,NEUTRAL,RESTED/RELAXED,SAD,TENSE/ANXIOUS,TIRED,ENTERTAINMENT,GYM,HOME,HOME_OFFICE,OTHER,OUTDOORS,TRANSIT,WORK/SCHOOL
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,NaN,2351.59,62.79210,6517.5,['Walk'],71.701565,149.0,24.0,33.0,713.0,False,NaN,62.073070,31260000.0,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0,<30,MALE,<19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,NaN,2332.08,62.67912,7178.6,['Walk'],70.579300,132.0,25.0,31.0,704.0,False,NaN,62.121476,32880000.0,0.0,460.0,88.0,0.0,94.0,1.466667,1.142857,0.947566,1.197531,9727.0,56.0,1374.0,4.0,0.0,<30,MALE,<19,14999,10000.0,15000.0,10000-14999,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,NaN,2262.30,62.57307,6090.9,['Walk'],71.842573,112.0,27.0,31.0,710.0,False,NaN,62.263999,33600000.0,0.0,493.0,67.0,0.0,96.0,1.116883,0.858974,1.015038,1.670732,8253.0,85.0,1350.0,0.0,0.0,<30,MALE,<19,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,['LIFETIME_DISTANCE'],2325.10,62.47493,6653.1,['Walk'],71.725477,133.0,21.0,37.0,622.0,False,NaN,62.368900,37620000.0,0.0,540.0,87.0,0.0,93.0,1.128205,1.129870,1.191729,1.588235,9015.0,90.0,1282.0,0.0,0.0,<30,MALE,<19,14999,10000.0,15000.0,10000-14999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,NaN,2586.76,62.41166,9557.9,['Walk'],74.401028,136.0,42.0,54.0,647.0,False,NaN,61.965409,33660000.0,0.0,493.0,68.0,0.0,94.0,0.910256,0.871795,1.211896,1.090909,12949.0,146.0,1274.0,4.0,0.0,<30,MALE,<19,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,621e362467b776a2404ad513,2021-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7406,621e36f967b776a240e5e7c9,2021-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7407,621e362467b776a2404ad513,2021-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7408,621e339967b776a240e502de,2021-11-19,NaN,NaN,NaN,N

In [11]:
# need to join the aggregated hourly data to the daily data here to get a full dataset for us to use. 
to_use = ['id', 'date', 'nremhr', 'rmssd', 'spo2', 'stress_score', 'sleep_points_percentage',
          'exertion_points_percentage', 'responsiveness_points_percentage', 'distance', 'activityType',
          'bpm', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes',
          'mindfulness_session', 'sleep_duration', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'gender',
          'bmi', 'TENSE/ANXIOUS', 'TIRED', "GYM", "HOME", "OUTDOORS"]
d = daily[to_use]
d

,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS
0,621e2e8e67b776a24055b564,2021-05-24,57.432,89.603,NaN,78.0,0.833333,0.675,0.866667,6517.5,['Walk'],71.701565,149.0,24.0,33.0,713.0,False,31260000.0,445.0,76.0,93.0,MALE,<19,NaN,NaN,NaN,NaN,NaN
1,621e2e8e67b776a24055b564,2021-05-25,57.681,94.303,NaN,80.0,0.833333,0.725,0.866667,7178.6,['Walk'],70.579300,132.0,25.0,31.0,704.0,False,32880000.0,460.0,88.0,94.0,MALE,<19,0.0,1.0,0.0,0.0,1.0
2,621e2e8e67b776a24055b564,2021-05-26,57.481,119.212,NaN,84.0,0.966667,0.725,0.866667,6090.9,['Walk'],71.842573,112.0,27.0,31.0,710.0,False,33600000.0,493.0,67.0,96.0,MALE,<19,0.0,0.0,0.0,0.0,1.0
3,621e2e8e67b776a24055b564,2021-05-27,57.493,111.709,NaN,82.0,0.933333,0.725,0.833333,6653.1,['Walk'],71.725477,133.0,21.0,37.0,622.0,False,37620000.0,540.0,87.0,93.0,MALE,<19,0.0,0.0,0.0,1.0,1.0
4,621e2e8e67b776a24055b564,2021-05-28,56.750,103.034,NaN,81.0,0.866667,0.725,0.866667,9557.9,['Walk'],74.401028,136.0,42.0,54.0,647.0,False,33660000.0,493.0,68.0,94.0,MALE,<19,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,621e362467b776a2404ad513,2021-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0
7406,621e36f967b776a240e5e7c9,2021-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0
7407,621e362467b776a2404ad513,2021-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
7408,621e339967b776a240e502de,2021-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0


In [12]:
nan_values = d.isna()
nan_values = nan_values.reset_index()
true_nan_values = nan_values.sum()
true_nan_values

index                               27450345
id                                         0
date                                       0
nremhr                                  4935
rmssd                                   4935
spo2                                    6140
stress_score                            5534
sleep_points_percentage                 5534
exertion_points_percentage              5534
responsiveness_points_percentage        5534
distance                                2633
activityType                            5197
bpm                                     2606
lightly_active_minutes                   327
moderately_active_minutes                327
very_active_minutes                      327
sedentary_minutes                        327
mindfulness_session                      299
sleep_duration                          3859
minutesAsleep                           3859
minutesAwake                            3859
sleep_efficiency                        3859
gender    

In [13]:
median_values_impute = ["nremhr", "rmssd", "spo2", "stress_score", "sleep_points_percentage", 
                        "exertion_points_percentage", "responsiveness_points_percentage", "distance", 
                        "minutesAsleep", "minutesAwake", "sleep_efficiency", "bpm", 
                        "lightly_active_minutes", "moderately_active_minutes", "very_active_minutes"]
d[median_values_impute] = d[median_values_impute].astype(np.float64)

median_values = d.groupby('id')[median_values_impute].median()

d[median_values_impute] = d[median_values_impute].fillna(median_values)
d = d.fillna(0)
d

/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_28262/1636732420.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[median_values_impute] = d[median_values_impute].astype(np.float64)
/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_28262/1636732420.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[median_values_impute] = d[median_values_impute].fillna(median_values)


,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS
0,621e2e8e67b776a24055b564,2021-05-24,57.432,89.603,0.0,78.0,0.833333,0.675,0.866667,6517.5,['Walk'],71.701565,149.0,24.0,33.0,713.0,False,31260000.0,445.0,76.0,93.0,MALE,<19,0.0,0.0,0.0,0.0,0.0
1,621e2e8e67b776a24055b564,2021-05-25,57.681,94.303,0.0,80.0,0.833333,0.725,0.866667,7178.6,['Walk'],70.579300,132.0,25.0,31.0,704.0,False,32880000.0,460.0,88.0,94.0,MALE,<19,0.0,1.0,0.0,0.0,1.0
2,621e2e8e67b776a24055b564,2021-05-26,57.481,119.212,0.0,84.0,0.966667,0.725,0.866667,6090.9,['Walk'],71.842573,112.0,27.0,31.0,710.0,False,33600000.0,493.0,67.0,96.0,MALE,<19,0.0,0.0,0.0,0.0,1.0
3,621e2e8e67b776a24055b564,2021-05-27,57.493,111.709,0.0,82.0,0.933333,0.725,0.833333,6653.1,['Walk'],71.725477,133.0,21.0,37.0,622.0,False,37620000.0,540.0,87.0,93.0,MALE,<19,0.0,0.0,0.0,1.0,1.0
4,621e2e8e67b776a24055b564,2021-05-28,56.750,103.034,0.0,81.0,0.866667,0.725,0.866667,9557.9,['Walk'],74.401028,136.0,42.0,54.0,647.0,False,33660000.0,493.0,68.0,94.0,MALE,<19,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,621e362467b776a2404ad513,2021-05-18,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,1.0,0.0
7406,621e36f967b776a240e5e7c9,2021-05-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,1.0,0.0
7407,621e362467b776a2404ad513,2021-05-23,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
7408,621e339967b776a240e502de,2021-11-19,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,1.0,0.0


In [14]:
d.to_csv("~/class/f24/cbm/evo-life/data/prepped/daily.csv")

OSError: Cannot save file into a non-existent directory: '/Users/jacksonmiskill/class/f24/cbm/evo-life/data/prepped'

In [15]:
# find out how many days each id has
days_per_id = d.groupby('id').agg(
    {
        "date": ['count']
    }
)
print("Total number of IDs", days_per_id.__len__())
days_per_id

Total number of IDs 71


,date
,count
id,
621e2e8e67b776a24055b564,71
621e2eaf67b776a2406b14ac,88
621e2ed667b776a24085d8d1,114
621e2ef567b776a24099f889,100
621e2efa67b776a2409dd1c3,137
621e2f1b67b776a240b3d87c,236
621e2f3967b776a240c654db,70
621e2f5767b776a240d8f9d6,72


In [16]:
# convert dates to numbered days
d = d.sort_values(by="date")
d['day'] = d.groupby("id").cumcount()
d.loc[d['id']=="621e346f67b776a24081744f"].sort_values(by="day")


,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
5251,621e346f67b776a24081744f,2021-05-24,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,454.2,0,84.097914,46.0,0.0,0.0,1394.0,False,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0
5252,621e346f67b776a24081744f,2021-05-25,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,1528.5,0,80.780103,138.0,0.0,0.0,755.0,False,32820000.0,488.0,59.0,97.0,0,0,0.0,0.0,0.0,0.0,0.0,1
5253,621e346f67b776a24081744f,2021-05-26,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,1953.5,0,80.951837,143.0,0.0,0.0,761.0,False,32160000.0,467.0,69.0,98.0,0,0,0.0,0.0,0.0,0.0,0.0,2
5254,621e346f67b776a24081744f,2021-05-27,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,1256.3,0,80.378686,81.0,0.0,0.0,828.0,False,31860000.0,466.0,65.0,94.0,0,0,0.0,0.0,0.0,0.0,0.0,3
5255,621e346f67b776a24081744f,2021-05-28,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,6038.7,['Walk'],85.813545,184.0,43.0,6.0,1207.0,False,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5490,621e346f67b776a24081744f,2022-01-18,70.490,23.205,0.0,67.0,0.800000,0.525,0.733333,2482.7,0,83.023664,120.0,0.0,0.0,817.0,False,30180000.0,448.0,55.0,93.0,0,0,0.0,0.0,0.0,0.0,0.0,239
5491,621e346f67b776a24081744f,2022-01-19,83.002,16.719,0.0,65.0,0.833333,0.525,0.633333,2743.9,0,83.768071,142.0,0.0,0.0,787.0,False,30660000.0,440.0,71.0,93.0,0,0,0.0,0.0,0.0,0.0,0.0,240
5492,621e346f67b776a24081744f,2022-01-20,71.996,25.288,0.0,71.0,0.866667,0.550,0.766667,3358.8,0,83.866642,172.0,0.0,0.0,706.0,False,33720000.0,488.0,74.0,93.0,0,0,0.0,0.0,0.0,0.0,0.0,241
5493,621e346f67b776a24081744f,2022-01-21,76.917,21.818,0.0,72.0,0.900000,0.550,0.766667,1753.1,0,80.768148,108.0,0.0,0.0,797.0,False,32100000.0,476.0,59.0,91.0,0,0,0.0,0.0,0.0,0.0,0.0,242


In [17]:
number_records_per_id_breq = breq.groupby("user_id").agg(
    {
        "breq_introjected_regulation": "count"
    }
)
number_records_per_id_breq.count()

breq_introjected_regulation    52
dtype: int64

In [18]:
breq['user_id'].count()

np.int64(92)

In [19]:
print("Number of records in panas:", panas['user_id'].count())
number_records_per_id_panas = panas.groupby("user_id").agg(
    {
        "positive_affect_score": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_panas.count())
number_records_per_id_panas

Number of records in panas: 268
Number of individual ids represented: positive_affect_score    51
dtype: int64


,positive_affect_score
user_id,
621e2e8e67b776a24055b564,5
621e2eaf67b776a2406b14ac,6
621e2ed667b776a24085d8d1,5
621e2f3967b776a240c654db,9
621e2f6167b776a240e082a9,7
621e2f7a67b776a240f14425,9
621e2f9167b776a240011ccb,5
621e2fb367b776a24015accd,8
621e2fce67b776a240279baa,5


In [20]:
personality.columns

Index(['Unnamed: 0', 'user_id', 'type', 'submitdate', 'extraversion',
       'agreeableness', 'conscientiousness', 'stability', 'intellect',
       'gender', 'ipip_extraversion_category', 'ipip_agreeableness_category',
       'ipip_conscientiousness_category', 'ipip_stability_category',
       'ipip_intellect_category'],
      dtype='object')

In [21]:
print("Number of records in personality:", personality['user_id'].count())
number_records_per_id_personality = personality.groupby("user_id").agg(
    {
        "extraversion": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_personality.count())
number_records_per_id_personality

Number of records in personality: 50
Number of individual ids represented: extraversion    50
dtype: int64


,extraversion
user_id,
621e2e8e67b776a24055b564,1
621e2eaf67b776a2406b14ac,1
621e2ed667b776a24085d8d1,1
621e2f3967b776a240c654db,1
621e2f6167b776a240e082a9,1
621e2f7a67b776a240f14425,1
621e2f9167b776a240011ccb,1
621e2fb367b776a24015accd,1
621e2fce67b776a240279baa,1


In [ ]:
stai.columns

In [ ]:
print("Number of records in stai:", stai['user_id'].count())
number_records_per_id_stai = stai.groupby("user_id").agg(
    {
        "stai_stress": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_stai.count())
number_records_per_id_stai

In [ ]:
ttm.columns

In [ ]:

print("Number of records in ttm:", ttm['user_id'].count())
number_records_per_id_ttm = ttm.groupby("user_id").agg(
    {
        "ttm_consciousness_raising": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_ttm.count())
number_records_per_id_ttm

In [ ]:
# let's take subsets of each of the behavioral tables and then join them together so that we have
# one table that contains behavioral information on each person who took the surveys

# problem: some people have multiple records in the behavior tables. Do I want to average those values together

# my decision is to do an average for each value

breq_averaged = breq.groupby("user_id").agg(
    {
        "breq_amotivation":"mean",
        "breq_external_regulation": "mean", 
        "breq_introjected_regulation": "mean", 
        "breq_identified_regulation": "mean", 
        "breq_intrinsic_regulation": "mean", 
        "breq_self_determination": "first"
    }
)
breq_averaged = breq_averaged.reset_index()
print("BREQ ID COUNT:", breq_averaged['user_id'].count())

panas_averaged = panas.groupby("user_id").agg(
    {
        "positive_affect_score":"mean",
        "negative_affect_score": "mean"
    }
)
panas_averaged = panas_averaged.reset_index()
print("PANAS ID COUNT:", panas_averaged['user_id'].count())

personality_averaged = personality.groupby("user_id").agg(
    {
        "agreeableness": "mean",
        "conscientiousness": "mean",
        "stability": "mean", 
        "intellect": "mean"
    }
)

personality_averaged = personality_averaged.reset_index()
print("PERSONALITY ID COUNT:", personality_averaged['user_id'].count())

stai_averaged = stai.groupby("user_id").agg(
    {
        "stai_stress": "mean",
        "stai_stress_category" : "first"
    }
)

stai_averaged = stai_averaged.reset_index()
print("STAI ID COUNT:", stai_averaged['user_id'].count())

ttm_averaged = ttm.groupby("user_id").agg(
    {
        "ttm_consciousness_raising": "mean", 
        "ttm_dramatic_relief": "mean", 
        "ttm_environmental_reevaluation": "mean", 
        "ttm_self_reevaluation": "mean", 
        "ttm_social_liberation": "mean", 
        "ttm_counterconditioning": "mean", 
        "ttm_helping_relationships": "mean",
        "ttm_reinforcement_management": "mean",
        "ttm_self_liberation": "mean", 
        "ttm_stimulus_control": "mean"
    }
)

ttm_averaged = ttm_averaged.reset_index()
print("TTM ID COUNT:", ttm_averaged['user_id'].count())




BREQ ID COUNT: 52
PANAS ID COUNT: 51
PERSONALITY ID COUNT: 50
STAI ID COUNT: 53
TTM ID COUNT: 53


In [ ]:
step_one = breq_averaged.merge(panas_averaged, on='user_id', how='outer')
step_two = step_one.merge(personality_averaged, on='user_id', how='outer')
step_three = step_two.merge(stai_averaged, on='user_id', how='outer')
individuals = step_three.merge(ttm_averaged, on='user_id', how='outer')
individuals

,user_id,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination,positive_affect_score,negative_affect_score,agreeableness,...,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,ttm_reinforcement_management,ttm_self_liberation,ttm_stimulus_control
0,621e2e8e67b776a24055b564,1.000000,1.000000,1.333333,3.500000,4.125000,intrinsic_regulation,36.000000,12.400000,33.0,...,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,4.000000,4.000000,1.666667
1,621e2eaf67b776a2406b14ac,1.125000,1.125000,2.000000,4.250000,3.625000,identified_regulation,28.666667,20.333333,45.0,...,3.500000,2.500000,4.166667,4.500000,3.666667,2.000000,3.000000,4.333333,3.333333,2.000000
2,621e2ed667b776a24085d8d1,1.875000,1.750000,2.666667,2.500000,1.750000,introjected_regulation,29.200000,24.400000,43.0,...,1.333333,2.166667,3.500000,3.333333,3.500000,1.166667,1.000000,2.833333,2.000000,1.333333
3,621e2f1b67b776a240b3d87c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2f3967b776a240c654db,1.000000,2.000000,1.000000,2.500000,3.000000,intrinsic_regulation,29.000000,19.666667,34.0,...,2.000000,2.000000,2.666667,4.000000,5.000000,2.333333,2.000000,3.000000,2.333333,2.000000
5,621e2f6167b776a240e082a9,1.000000,1.375000,3.166667,4.875000,4.750000,identified_regulation,24.428571,24.571429,41.0,...,2.500000,4.166667,2.833333,5.000000,2.500000,2.166667,2.333333,4.500000,4.666667,3.833333
6,621e2f7a67b776a240f14425,1.000000,1.166667,1.111111,3.416667,3.666667,intrinsic_regulation,35.333333,17.888889,38.0,...,2.000000,1.500000,2.000000,4.666667,2.333333,3.333333,1.333333,4.500000,3.166667,2.166667
7,621e2f9167b776a240011ccb,1.000000,1.000000,4.166667,4.000000,5.000000,intrinsic_regulation,39.000000,33.400000,45.0,...,3.333333,2.833333,1.833333,4.833333,2.166667,1.666667,1.833333,4.833333,3.500000,4.166667
8,621e2fb367b776a24015accd,1.125000,1.625000,3.166667,3.250000,3.375000,intrinsic_regulation,31.000000,19.125000,36.0,...,1.833333,2.333333,2.333333,3.833333,3.500000,3.333333,4.000000,3.833333,3.833333,2.666667
9,621e2fce67b776a240279baa,1.000000,2.500000,3.000000,2.500000,4.500000,intrinsic_regulation,26.400000,17.600000,42.0,...,2.666667,2.666667,2.833333,4.500000,4.500000,1.333333,3.166667,3.833333,2.333333,2.666667


In [ ]:
individuals[individuals.select_dtypes(include=['float']).columns] = individuals.select_dtypes(include=['float']).fillna(0)
individuals[individuals.select_dtypes(include=['object']).columns] = individuals.select_dtypes(include=['object']).fillna('undefined')
individuals

,user_id,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination,positive_affect_score,negative_affect_score,agreeableness,...,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,ttm_reinforcement_management,ttm_self_liberation,ttm_stimulus_control
0,621e2e8e67b776a24055b564,1.000000,1.000000,1.333333,3.500000,4.125000,intrinsic_regulation,36.000000,12.400000,33.0,...,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,4.000000,4.000000,1.666667
1,621e2eaf67b776a2406b14ac,1.125000,1.125000,2.000000,4.250000,3.625000,identified_regulation,28.666667,20.333333,45.0,...,3.500000,2.500000,4.166667,4.500000,3.666667,2.000000,3.000000,4.333333,3.333333,2.000000
2,621e2ed667b776a24085d8d1,1.875000,1.750000,2.666667,2.500000,1.750000,introjected_regulation,29.200000,24.400000,43.0,...,1.333333,2.166667,3.500000,3.333333,3.500000,1.166667,1.000000,2.833333,2.000000,1.333333
3,621e2f1b67b776a240b3d87c,0.000000,0.000000,0.000000,0.000000,0.000000,undefined,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,621e2f3967b776a240c654db,1.000000,2.000000,1.000000,2.500000,3.000000,intrinsic_regulation,29.000000,19.666667,34.0,...,2.000000,2.000000,2.666667,4.000000,5.000000,2.333333,2.000000,3.000000,2.333333,2.000000
5,621e2f6167b776a240e082a9,1.000000,1.375000,3.166667,4.875000,4.750000,identified_regulation,24.428571,24.571429,41.0,...,2.500000,4.166667,2.833333,5.000000,2.500000,2.166667,2.333333,4.500000,4.666667,3.833333
6,621e2f7a67b776a240f14425,1.000000,1.166667,1.111111,3.416667,3.666667,intrinsic_regulation,35.333333,17.888889,38.0,...,2.000000,1.500000,2.000000,4.666667,2.333333,3.333333,1.333333,4.500000,3.166667,2.166667
7,621e2f9167b776a240011ccb,1.000000,1.000000,4.166667,4.000000,5.000000,intrinsic_regulation,39.000000,33.400000,45.0,...,3.333333,2.833333,1.833333,4.833333,2.166667,1.666667,1.833333,4.833333,3.500000,4.166667
8,621e2fb367b776a24015accd,1.125000,1.625000,3.166667,3.250000,3.375000,intrinsic_regulation,31.000000,19.125000,36.0,...,1.833333,2.333333,2.333333,3.833333,3.500000,3.333333,4.000000,3.833333,3.833333,2.666667
9,621e2fce67b776a240279baa,1.000000,2.500000,3.000000,2.500000,4.500000,intrinsic_regulation,26.400000,17.600000,42.0,...,2.666667,2.666667,2.833333,4.500000,4.500000,1.333333,3.166667,3.833333,2.333333,2.666667


In [ ]:
individuals.to_csv("~/class/f24/cbm/evo-life/data/prepped/individuals.csv")

In [ ]:
d.sort_values(by=['id', 'date'])

,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,...,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
0,621e2e8e67b776a24055b564,2021-05-24,57.432,89.603,0.0,78.0,0.833333,0.675,0.866667,6517.5,...,76.0,93.0,MALE,<19,0.0,0.0,0.0,0.0,0.0,0
1,621e2e8e67b776a24055b564,2021-05-25,57.681,94.303,0.0,80.0,0.833333,0.725,0.866667,7178.6,...,88.0,94.0,MALE,<19,0.0,1.0,0.0,0.0,1.0,1
2,621e2e8e67b776a24055b564,2021-05-26,57.481,119.212,0.0,84.0,0.966667,0.725,0.866667,6090.9,...,67.0,96.0,MALE,<19,0.0,0.0,0.0,0.0,1.0,2
3,621e2e8e67b776a24055b564,2021-05-27,57.493,111.709,0.0,82.0,0.933333,0.725,0.833333,6653.1,...,87.0,93.0,MALE,<19,0.0,0.0,0.0,1.0,1.0,3
4,621e2e8e67b776a24055b564,2021-05-28,56.750,103.034,0.0,81.0,0.866667,0.725,0.866667,9557.9,...,68.0,94.0,MALE,<19,0.0,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7395,621e375b67b776a240290cdc,2021-08-13,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,...,0.0,0.0,FEMALE,23.0,0.0,0.0,0.0,0.0,0.0,81
7396,621e375b67b776a240290cdc,2021-08-14,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,...,0.0,0.0,FEMALE,23.0,0.0,0.0,0.0,0.0,0.0,82
7397,621e375b67b776a240290cdc,2021-08-15,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,...,0.0,0.0,FEMALE,23.0,0.0,0.0,0.0,0.0,0.0,83
7398,621e375b67b776a240290cdc,2021-08-16,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,...,0.0,0.0,FEMALE,23.0,0.0,0.0,0.0,0.0,0.0,84


In [ ]:
d.head(100)

,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
6368,621e362467b776a2404ad513,2021-04-08,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0
6369,621e362467b776a2404ad513,2021-04-09,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1
6370,621e362467b776a2404ad513,2021-04-10,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,2
6371,621e362467b776a2404ad513,2021-04-12,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,3
6372,621e362467b776a2404ad513,2021-04-13,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,4
6373,621e362467b776a2404ad513,2021-04-16,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,5
6374,621e362467b776a2404ad513,2021-04-17,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,6
6375,621e362467b776a2404ad513,2021-04-19,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,7
6376,621e362467b776a2404ad513,2021-04-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,8
6377,621e362467b776a2404ad513,2021-04-21,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,9


In [ ]:
# going to need to filter down the dataframe to the participants that actually had good data
tmp = daily.loc[daily['nightly_temperature'] > 0 ]
tmp['id'].nunique()

64

In [ ]:
d.apply(lambda row: (row == 0).sum(), axis=1)

6368    27
6369    26
6370    26
6371    26
6372    26
        ..
3498    23
1908    23
2535    23
2081    23
3757    14
Length: 7410, dtype: int64

In [ ]:
d.tail(100)

,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
6254,621e356967b776a24027bd9f,2022-01-19,59.473,56.795,0.0,0.0,0.000000,0.000,0.000000,1382.8,0,73.111696,142.0,0.0,0.0,753.0,False,32700000.0,483.0,62.0,95.0,0,0,0.0,0.0,0.0,0.0,0.0,65
4154,621e333967b776a240a3cd06,2022-01-19,57.838,87.402,96.2,0.0,0.000000,0.000,0.000000,89.9,0,62.301975,11.0,0.0,0.0,889.0,False,32400000.0,478.0,62.0,95.0,MALE,21.0,0.0,0.0,0.0,0.0,0.0,104
2010,621e309b67b776a240b532b0,2022-01-19,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,7560.8,0,71.209404,82.0,15.0,56.0,602.0,False,27180000.0,398.0,55.0,98.0,MALE,23.0,0.0,0.0,0.0,0.0,0.0,101
156,621e2eaf67b776a2406b14ac,2022-01-20,73.883,24.163,0.0,76.0,0.833333,0.775,0.666667,17652.5,"['Aerobic Workout', 'Sport', 'Walk']",88.184901,437.0,65.0,57.0,297.0,False,35040000.0,493.0,91.0,90.0,FEMALE,<19,0.0,0.0,0.0,0.0,0.0,85
2533,621e310d67b776a24003096d,2022-01-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,FEMALE,20.0,0.0,0.0,0.0,0.0,0.0,107
2642,621e312a67b776a240164d59,2022-01-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,MALE,25.0,0.0,0.0,0.0,0.0,0.0,106
3496,621e32d067b776a2405b7d54,2022-01-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,FEMALE,22.0,0.0,0.0,0.0,0.0,0.0,82
2993,621e323667b776a240f19134,2022-01-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,MALE,19.0,0.0,0.0,0.0,0.0,0.0,85
3755,621e32e667b776a2406d2f1c,2022-01-20,76.127,24.956,93.1,68.0,0.833333,0.575,0.666667,3128.4,0,83.727662,177.0,0.0,0.0,877.0,False,23160000.0,348.0,38.0,96.0,MALE,<19,0.0,0.0,0.0,0.0,0.0,94
2011,621e309b67b776a240b532b0,2022-01-20,0.000,0.000,0.0,0.0,0.000000,0.000,0.000000,0.0,0,0.000000,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,MALE,23.0,0.0,0.0,0.0,0.0,0.0,102


In [22]:
from helper import GenomeCreator

gc = GenomeCreator()
path_to_daily_csv = "~/class/f24/cbm/evo-life/prepped/daily.csv"
gc.load_data_from_csv(path = path_to_daily_csv)



In [25]:
gc.determine_quality_individual(id = "621e32e667b776a2406d2f1c", quality = 0.3)

True

In [35]:
final_string = ""
i = 0
for c in list(d.columns):
    # if i % 5 == 0:
    #     final_string += '\n'
    # i += 1
    final_string += c + " "
final_string

'id date nremhr rmssd spo2 stress_score sleep_points_percentage exertion_points_percentage responsiveness_points_percentage distance activityType bpm lightly_active_minutes moderately_active_minutes very_active_minutes sedentary_minutes mindfulness_session sleep_duration minutesAsleep minutesAwake sleep_efficiency gender bmi TENSE/ANXIOUS TIRED GYM HOME OUTDOORS day '

In [29]:
list(d['id'].unique())

['621e362467b776a2404ad513',
 '621e36f967b776a240e5e7c9',
 '621e332267b776a24092a584',
 '621e30e467b776a240e817c7',
 '621e367e67b776a24087d75d',
 '621e360b67b776a24039709f',
 '621e2f6167b776a240e082a9',
 '621e2f1b67b776a240b3d87c',
 '621e32af67b776a24045b4cf',
 '621e2f9167b776a240011ccb',
 '621e301e67b776a240608a72',
 '621e337667b776a240ce78ab',
 '621e2ed667b776a24085d8d1',
 '621e351a67b776a240f6204b',
 '621e33ed67b776a2401cf5f7',
 '621e33b067b776a240f39e56',
 '621e375b67b776a240290cdc',
 '621e30c867b776a240d4aa6c',
 '621e36bb67b776a240b40d64',
 '621e2f7a67b776a240f14425',
 '621e34ca67b776a240be3b69',
 '621e34ec67b776a240d60873',
 '621e341067b776a24037b105',
 '621e2e8e67b776a24055b564',
 '621e34f767b776a240de4e1a',
 '621e345c67b776a2407146a8',
 '621e314867b776a24029ebf9',
 '621e346f67b776a24081744f',
 '621e36dd67b776a240ce9a45',
 '621e2fce67b776a240279baa',
 '621e34ff67b776a240e446d6',
 '621e326767b776a24012e179',
 '621e2f3967b776a240c654db',
 '621e34db67b776a240c9c2be',
 '621e36c267b7